<a href="https://colab.research.google.com/github/Ryan-Andrade/PySpark/blob/main/Stop_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()


In [ ]:
# Create DataFrame
sentenceData = spark.createDataFrame([
    (0, ["Big", "data", "is", "super", "powerful"]),
    (1, ["This", "is", "going", "to", "be", "epic"])                                  
], ["id", "raw"])

sentenceData.show(truncate=False)

+---+--------------------------------+
|id |raw                             |
+---+--------------------------------+
|0  |[Big, data, is, super, powerful]|
|1  |[This, is, going, to, be, epic] |
+---+--------------------------------+



In [ ]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover

In [ ]:
# Run the Remover
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

In [ ]:
# Transform and show data
remover.transform(sentenceData).show(truncate=False)

+---+--------------------------------+----------------------------+
|id |raw                             |filtered                    |
+---+--------------------------------+----------------------------+
|0  |[Big, data, is, super, powerful]|[Big, data, super, powerful]|
|1  |[This, is, going, to, be, epic] |[going, epic]               |
+---+--------------------------------+----------------------------+



 Combine both tokenizer and StopWordsRemover on a DataFrame that isn't already broken out into a list of words.


In [ ]:
# Import the Tokenizer library
from pyspark.ml.feature import Tokenizer

In [ ]:
# Create sample DataFrame
dataframe = spark.createDataFrame([
    (0, "I am learning"),
    (1, "how to do NLP processes"),
    (2, "to improve my Machine Learning skills")                               
], ["id", "sentence"])
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|       I am learning|
|  1|how to do NLP pro...|
|  2|to improve my Mac...|
+---+--------------------+



In [ ]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_caeb0cb8d5c0

In [ ]:
# Transform and show DataFrame
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)

+---+-------------------------------------+--------------------------------------------+
|id |sentence                             |words                                       |
+---+-------------------------------------+--------------------------------------------+
|0  |I am learning                        |[i, am, learning]                           |
|1  |how to do NLP processes              |[how, to, do, nlp, processes]               |
|2  |to improve my Machine Learning skills|[to, improve, my, machine, learning, skills]|
+---+-------------------------------------+--------------------------------------------+



In [ ]:
# Run the Remover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [ ]:
# Transform and show data
tokenized_df = remover.transform(tokenized_df).show(truncate=False)

+---+-------------------------------------+--------------------------------------------+------------------------------------+
|id |sentence                             |words                                       |filtered                            |
+---+-------------------------------------+--------------------------------------------+------------------------------------+
|0  |I am learning                        |[i, am, learning]                           |[learning]                          |
|1  |how to do NLP processes              |[how, to, do, nlp, processes]               |[nlp, processes]                    |
|2  |to improve my Machine Learning skills|[to, improve, my, machine, learning, skills]|[improve, machine, learning, skills]|
+---+-------------------------------------+--------------------------------------------+------------------------------------+

